# Multiple Regression

- Do age and IQ scores effectively predict GPA?
- Do weight, height, and age explain the variance in cholesterol levels?

## Learning goals:

For a multivariable linear regression, students will be able to:

* compare and contrast with univariable linear regression
* write an example of the equation
* develop one with statsmodels 
* assess the model fit 
* interpret coefficients
* validate the model
* export the model

### Keyterms
- Multivariable
- Train-test split
- MSE: Mean squared error
- RSME: Root squared mean error

## Scenario

The University of San Paulo in Brazil is likes to party. We are a contracted beer supplier to the University and we want to make sure we have enough supply on hand. We are hoping to build a model that can predict beer consumption given other variables. 


![beer](pexels-photo-544988-small.jpeg)
More about the dataset can be found [here](https://www.kaggle.com/dongeorge/beer-consumption-sao-paulo)

###  Prior Knowledge


Before looking at the dataset, what variables do we think might be in there? What might make a student drink more? 

#### Step 1:  Discussion 

- compare and contrast with univariable linear regression
- How is this different from the regression we've done before?
- Here, you'll explore how to perform linear regressions using multiple independent variables to better predict a target variable.

#### Step 2:  Develop a multivariable regression model with statsmodels 

**Load Libraries and load in data**

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [2]:
df = pd.read_csv('Consumo_cerveja.csv')

In [3]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 7 columns):
Data                           365 non-null object
Temperatura Media (C)          365 non-null object
Temperatura Minima (C)         365 non-null object
Temperatura Maxima (C)         365 non-null object
Precipitacao (mm)              365 non-null object
Final de Semana                365 non-null float64
Consumo de cerveja (litros)    365 non-null float64
dtypes: float64(2), object(5)
memory usage: 51.5+ KB


### Small Data Cleaning Tasks:
- Drop Date
- convert all the columns to numeric (replace ',' with '.')
- rename columns to be `name = ['temp-median', 'temp-min', 'temp-max', 'rain', 'finals-week', 'target']`

In [4]:
# clean data here
df = df.drop(columns=['Data'])

df = df.applymap(lambda x: x.replace(',','.') if type(x)==str else x)
df.head()

,Temperatura Media (C),Temperatura Minima (C),Temperatura Maxima (C),Precipitacao (mm),Final de Semana,Consumo de cerveja (litros)
0,27.3,23.9,32.5,0,0.0,25.461
1,27.02,24.5,33.5,0,0.0,28.972
2,24.82,22.4,29.9,0,1.0,30.814
3,23.98,21.5,28.6,1.2,1.0,29.799
4,23.82,21,28.3,0,0.0,28.900


In [5]:
names = ['temp_median', 'temp_min', 'temp_max', 'rain', 'finals_week', 'target']
df.columns = names
# df = df.applymap(lambda x:x.replace(',','.') if type(x)==str else x)
# always assign to a variable 
df.dtypes

temp_median     object
temp_min        object
temp_max        object
rain            object
finals_week    float64
target         float64
dtype: object

In [6]:
df['temp_median'] = df['temp_median'].astype(float)
df['temp_min'] = df['temp_min'].astype(float)
df['temp_max'] = df['temp_max'].astype(float)
df['rain'] = df['rain'].astype(float)
# df['temp-median'] df['temp-median'].astype(float)
# df.dtypes

In [7]:
df.info()
# df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 6 columns):
temp_median    365 non-null float64
temp_min       365 non-null float64
temp_max       365 non-null float64
rain           365 non-null float64
finals_week    365 non-null float64
target         365 non-null float64
dtypes: float64(6)
memory usage: 44.2 KB


**Check** for NaNs

In [8]:
df.isna().sum()

temp_median    576
temp_min       576
temp_max       576
rain           576
finals_week    576
target         576
dtype: int64

In [9]:
df.dropna(inplace=True)

In [10]:
df.shape

(365, 6)

### Everyone write an example of an equation for our multiple regression

The main idea here is pretty simple. Whereas, in simple linear regression we took our dependent variable to be a function only of a single independent variable, here we'll be taking the dependent variable to be a function of multiple independent variables.

<img src="https://miro.medium.com/max/1400/1*d0icRnPHWjHSNXxuoYT5Vg.png" width=450 />

Our regression equation, then, instead of looking like $\hat{y} = mx + b$, will now look like:

$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x_1 + ... + \hat{\beta}_nx_n$.

Remember that the hats ( $\hat{}$ ) indicate parameters that are estimated.

$$ \hat y = \hat\beta_0 + \hat\beta_1 x_1 + \hat\beta_2 x_2 +\ldots + \hat\beta_n x_n $$ 

What would the formula be with real values?

**Send your equations to me via zoom or slack and I will paste them into the notebook**

Equations here

>

![statsmodels](https://www.statsmodels.org/stable/_static/statsmodels_hybi_banner.png)

Okay, now here's how you can use format and join to make the formula with **code**:

In [11]:
print(df.columns)
df.head(3)


Index(['temp_median', 'temp_min', 'temp_max', 'rain', 'finals_week', 'target'], dtype='object')


,temp_median,temp_min,temp_max,rain,finals_week,target
0,27.30,23.9,32.5,0.0,0.0,25.461
1,27.02,24.5,33.5,0.0,0.0,28.972
2,24.82,22.4,29.9,0.0,1.0,30.814


In [12]:
type(df.target)

pandas.core.series.Series

In [13]:
df2 = df.drop(columns=['temp_median','temp_min'])
df2.columns

Index(['temp_max', 'rain', 'finals_week', 'target'], dtype='object')

In [14]:
formula = 'target~{}'.format("+".join(df.columns[:-1]))
print(formula)

target~temp_median+temp_min+temp_max+rain+finals_week


In [15]:
df.columns

Index(['temp_median', 'temp_min', 'temp_max', 'rain', 'finals_week', 'target'], dtype='object')

In [16]:
model = sm.OLS(df.target, df.drop('target', axis=1)).fit() # input the one dependent variable with all the indipendent variable 
print(model.summary()) # get the summary 

                            OLS Regression Results                            
Dep. Variable:                 target   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     7620.
Date:                Tue, 15 Oct 2019   Prob (F-statistic):               0.00
Time:                        11:34:44   Log-Likelihood:                -851.48
No. Observations:                 365   AIC:                             1713.
Df Residuals:                     360   BIC:                             1732.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
temp_median     0.1192      0.202      0.590      

In [ ]:
model.summary()

In [ ]:
# we decide to remove the two columens from our dataframe object
df2 = df.drop(columns=['temp_median','temp_min'])
df2.columns # = ['temp_max','rain','finals_week','target']

In [ ]:
# formula2 = 'target~{}'.format("+".join(df2.columns[:-1]))
formula2 = 'target~temp_max+rain+finals_week'
print(formula2)
df2.head(3)

In [ ]:
model2 = sm.OLS(df2.target, df2.drop('target', axis=1)).fit()
model2.summary()
# model = sm.OLS(df.target, df.drop('target', axis=1)).fit()

## **using statsmodels.formula.api**

In [ ]:
print(formula2)
df2.head(3)

In [ ]:
result = smf.ols(formula=formula2, data=df2).fit() # outcome/target in pridictor/fiture 
result.summary()
# df2.target, df2.drop('target', axis=1)).fit()

### What's the actual multivariable  linear regression equation with the coefficients?

$$ \hat y = \hat\beta_0 + \hat\beta_1 x_1 + \hat\beta_2 x_2 +\ldots + \hat\beta_n x_n $$ 

#### Step 3: Assess the model fit
Demonstrate and Apply:

**Discussion:**

In groups of 2 or 3 write a synopsis of the following summary

* What can you say about the coefficients?

* What do the p-values tell us?

* What does R^2 represent

* What other insights do you notice?





#### Step 4: Validate the model 
![scikit](https://cdn-images-1.medium.com/max/1200/1*-FHtcdQljtGKQGm77uDIyQ.png)
- Build LinReg Model with Scikit-Learn
- Check some of the linear regression assumptions


[Documentation for sklearn `LinearRegression()`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)

In [ ]:
linreg = LinearRegression()
df.head(3)

In [ ]:
X = df.drop("target", axis=1) # drop the target variable from the dataset -- this is our ML varible
y = df.target # creat our traget variable as a new dataframe to test

In [ ]:
X.head(3)

In [ ]:
y.head(3)

In [ ]:
print(type(y))
print(type(X))

#### Train test split
[sklearn function documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

#### So far we've used the whole dataset to build a model
![img1](whole_data.png)

#### But no promise how it will perform on new data

![img2](new_data.png)

#### So we split to help evaluate
![img3](tt_split.png)

In [ ]:
# use a train split linear regration package to train the model 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [ ]:
print(type(X_train))
print(type(y_train))


In [ ]:
X_train.head()

In [ ]:
# use fit to form model
linreg.fit(X_train, y_train)

In [ ]:
linreg.coef_

In [ ]:
linreg.intercept_

ß0 = 5.96
ß1 = 1.4
ß2 = 

In [ ]:
linreg = LinearRegression() # the same as below
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
linreg.fit(X_train, y_train) # the same as below
linreg.coef_
linreg.intercept_
linreg.score(X_test, y_test) # if the score is ~ 0.9, this is a really good model--gives you r squared of the model

# importing pickles 
import pickle
model = LinearRegression() # the same as above 
model.fit(X_train, y_train) # the same as above

with open('model.pkl','wb') as f:
    pickle.dump(model_save,f)
    
!ls # see where the pickled document is 
# use this to open the pickled document

with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

### Model evaluation

So far this looks very similar to `Statsmodels`.
Can you use the `LinearRegression` documentation to find:
- model coefficients?
- coefficients p-values?

In [ ]:
# gives you r squared of the model
linreg.score(X_test, y_test)

`score` here returns the R^2. 

How does it differ from when you use the whole dataset?

#### Saving model

![pickle](https://lovelygreens.com/wp-content/uploads/grandmas-dill-pickles-750x440.jpg)

```
model = LinearRegression()
model.fit(X_train, y_train)
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
```


then to reload later:

```
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)
```



In [ ]:
model_save = 


In [ ]:
import pickle
with open('model.pkl','wb') as f:
    pickle.dump(model_save,f)

In [ ]:
!ls

### Integration:

Repeat this process for concrete mixture. 
What combination of materials creates the strongest concrete compressive strength?

The documentation can be found [here](http://archive.ics.uci.edu/ml/datasets/concrete+compressive+strength)
![test](building-construction-building-site-constructing-small.jpg)

In [ ]:
con_df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/concrete/compressive/Concrete_Data.xls')

In [ ]:
# con_df.head()
# df4.info()
names = ['cement','blast_furnace_slag','fly_ash','water','superplasticizer','coarse_aggregate','fine_aggregate','age','concrete_compressive_strength']

con_df.columns = names
con_df.head(3)


In [ ]:
con_df['age'] = con_df['age'].astype(float)
con_df = con_df.dropna()

In [ ]:
con_df.dtypes

In [ ]:
X_con = con_df.drop("concrete_compressive_strength", axis=1)
y_con = con_df['concrete_compressive_strength']
print(type==)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_con, y_con, test_size=0.20)

In [ ]:
type(X_train)

In [ ]:
# use fit to form model
import pickle
con_model = LinearRegression() # the same as above 
con_model.fit(X_train, y_train)

In [ ]:
print(con_model.coef_)
print('The intercept is at:',con_model.intercept_)

In [ ]:
# gives you r squared of the model
con_model.score(X_test, y_test)

In [ ]:
# creating a pickle file
with open('construction_model.pkl', 'wb') as f:
    pickle.dump(con_model, f)
    
# check the location by running 
!ls


## Train another model with smaller set 

In [ ]:
# work on the smaller size 
# con_model2 = con_df.drop(columns=['blast_furnace_slag','fly_ash'], axis=1)
X_con2 = con_model2.drop(columns='concrete_compressive_strength')
y_con2 = con_model2['concrete_compressive_strength']

In [ ]:
# train the sample size before
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_con2, y_con2, test_size=0.20)

In [ ]:
# linear regret the result 
reg_model2 = LinearRegression()
reg_model2.fit(X_train2, y_train2)

# print the coeficents and the intercepts 

print('Model coeficents are', reg_model2.coef_)
print('The intercept is at:',reg_model2.intercept_)
reg_model2.score(X_test2, y_test2)


In [ ]:
y_con2.head()

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)
then to reload later:

with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

### Assessment

### Reflection

### Resources

Resources
https://towardsdatascience.com/linear-regression-detailed-view-ea73175f6e86

Full code implementation of Linear Regression
Full code — https://github.com/SSaishruthi/Linear_Regression_Detailed_Implementation

Multiple regression explained
https://www.statisticssolutions.com/what-is-multiple-linear-regression/
